# <font color= 'blue'> Deep Learning
# <font color='blue'> Project 1 </font>

## <font color= 'blue'> Fully Connected Neural Network
Mehek Niwas || MTH 4320 Fall 2023

Cannot use softmax with a binary classification.

Cannot calculate entropy for ReLU, therefore ReLU should not be the function for the output layer when loss must be calculated

Use k-fold cross-validation?

# <font color= 'blue'> Goals
*The goals of the project:*

1) Use a fully-connected feed forward neural network to predict the survival status of patients given a bone marrow transplant.

2) Calculation entropy for each layer of neural network for the best epoch, and calculate total entropy of each epoch

In [ ]:
import pandas as pd
!pip install ucimlrepo

# <font color= 'blue'> Dataset Preparation & Preprocessing
## Bone Marrow Transplants in Children

This dataset was obtained from from UCI Machine Learning Repository. Dataset has data on 187 bone transplant patients, with information such as DonorABO, Recipientage, PLTrecovery, and more in integer and 1-hot/binary form already.

The total number of patients with all data was 142, so the data from 45 patients had to be removed for a balanced dataset.

Other adjustments:
- switched "disease" categorial column to 1-hot encoding
- removed "ALL" diseases binary column due to redundancy
- removed "recipientage10" binary column due to redundancy with "recipientage" integer column
- removed "time to development of agvhd stage iii and iv" due to redundancy with "prescence of agvhd" column. values for time to development of agvhd were very skewed with the values to represent no development (ex: 1,000,000 days) as well.
- removed "survival time" integer column due to conflict with output for prediciting patient survival status


In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

# fetch dataset --> from https://archive.ics.uci.edu/dataset/565/bone+marrow+transplant+children
bone_marrow_transplant_children = fetch_ucirepo(id=565)

# metadata
print(bone_marrow_transplant_children.metadata)

# variable information
print(bone_marrow_transplant_children.variables)

{'uci_id': 565, 'name': 'Bone marrow transplant: children', 'repository_url': 'https://archive.ics.uci.edu/dataset/565/bone+marrow+transplant+children', 'data_url': 'https://archive.ics.uci.edu/static/public/565/data.csv', 'abstract': 'The data set describes pediatric patients with several hematologic diseases, who were subject to the unmanipulated allogeneic unrelated donor hematopoietic stem cell transplantation.', 'area': 'Health and Medicine', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 187, 'num_features': 36, 'feature_types': ['Integer', 'Real'], 'demographics': ['Gender', 'Age'], 'target_col': ['survival_status'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2020, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5NP6Z', 'creators': ['Marek Sikora', 'Łukasz Wróbel', 'Adam Gudyś'], 'intro_paper': {'title': 'GuideR: a guided separate-and-conquer rule learn

In [ ]:
{
    'uci_id': 20,
    'name': 'Census Income',
    'repository_url': 'https://archive.ics.uci.edu/dataset/20/census+income',
    'data_url': 'https://archive.ics.uci.edu/static/public/20/data.csv',
    'abstract': 'Predict whether income exceeds $50K/yr based on census data. Also known as Adult dataset.',
    'area': 'Social Science',
    'tasks': ['Classification'],
    'characteristics': ['Multivariate'],
    'num_instances': 48842,
    'num_features': 14,
    'feature_types': ['Categorical', 'Integer'],
    'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'],
    'target_col': ['income'],
    'index_col': None,
    'has_missing_values': 'yes',
    'missing_values_symbol': 'NaN',
    'year_of_dataset_creation': 1996,
    'last_updated': 'Thu Aug 10 2023',
    'dataset_doi': '10.24432/C5GP7S',
    'creators': ['Ron Kohavi'],
    'intro_paper': None,
    'additional_info': {
        'summary': 'Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))\r\n\r\nPrediction task is to determine whether a person makes over 50K a year.',
        'purpose': None,
        'funded_by': None,
        'instances_represent': None,
        'recommended_data_splits': None,
        'sensitive_data': None,
        'preprocessing_description': None,
        'variable_info': 'Listing of attributes:\r\n\r\n>50K, <=50K.\r\n\r\nage: continuous.\r\nworkclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.\r\nfnlwgt: continuous.\r\neducation: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.\r\neducation-num: continuous.\r\nmarital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.\r\noccupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.\r\nrelationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.\r\nrace: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.\r\nsex: Female, Male.\r\ncapital-gain: continuous.\r\ncapital-loss: continuous.\r\nhours-per-week: continuous.\r\nnative-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.',
        'citation': None
    }
}


{'uci_id': 20,
 'name': 'Census Income',
 'repository_url': 'https://archive.ics.uci.edu/dataset/20/census+income',
 'data_url': 'https://archive.ics.uci.edu/static/public/20/data.csv',
 'abstract': 'Predict whether income exceeds $50K/yr based on census data. Also known as Adult dataset.',
 'area': 'Social Science',
 'tasks': ['Classification'],
 'characteristics': ['Multivariate'],
 'num_instances': 48842,
 'num_features': 14,
 'feature_types': ['Categorical', 'Integer'],
 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'],
 'target_col': ['income'],
 'index_col': None,
 'has_missing_values': 'yes',
 'missing_values_symbol': 'NaN',
 'year_of_dataset_creation': 1996,
 'last_updated': 'Thu Aug 10 2023',
 'dataset_doi': '10.24432/C5GP7S',
 'creators': ['Ron Kohavi'],
 'intro_paper': None,
 'additional_info': {'summary': 'Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditi

In [ ]:
# data (as pandas dataframes)
X = bone_marrow_transplant_children.data.features
Y = bone_marrow_transplant_children.data.targets
print("Original Dataset: ")
print("Shape of Inputs: X =", X.shape)
print("Shape of Outputs: Y =", Y.shape)
print("")

# Convert the column name to lowercase to ensure case insensitivity
X.columns = X.columns.str.lower()

# code for dropping rows with any missing values
# Capturing the index before dropping rows from the input DataFrame
original_index = X.index

# Dropping rows with missing values from the input DataFrame
cleaned_X = X.dropna()

# Filtering corresponding rows from the output DataFrame
cleaned_Y = Y.loc[original_index.isin(cleaned_X.index)]
cleaned_X.reset_index(drop=True, inplace = True) # resetting weird indexes

# code for dropping identified columns upon data inspection by researcher
cleaned_X = cleaned_X.drop(columns=['survival_time'])
cleaned_X = cleaned_X.drop(['time_to_agvhd_iii_iv'], axis=1)
cleaned_X = cleaned_X.drop(['recipientage10'], axis=1)

cleaned_Y.reset_index(drop=True, inplace = True) # resetting weird indexes
print("Redudant and conflicting features: "); print(" -- time to development of agvhd stage iii and iv"); print(" -- survival time"); print(" -- recipientage10")
print("")
print("After removal of redundant and conflicting features, and patients with missing data: ")
print("New Shape of Inputs: X =", cleaned_X.shape)
print("New Shape of Outputs: Y =", cleaned_Y.shape)
print("")

# code for making the disease categories into 1 hot encoding

# Get one hot encoding & Drop column 'Disease'
one_hot = pd.get_dummies(cleaned_X['disease']) # i think the drop_first = True didnt work!!

# Join one_hot and to original dataframe (cleaned_X)
cleaned_X = cleaned_X.join(one_hot)


# code for ALL in disease category
# Define condition based on values in 'column_to_check'
condition = cleaned_X['disease'].isin(['ALL'])  # condition

# Modify values in specified columns based on the condition
cleaned_X.loc[condition, ['AML', 'chronic', 'lymphoma', 'nonmalignant']] = 1

cleaned_X = cleaned_X.drop('disease', axis=1)
cleaned_X = cleaned_X.drop('ALL', axis=1)

# making sure all columns are float type data!!
cleaned_X = cleaned_X.astype('float64')
cleaned_Y = cleaned_Y.astype('float64')

print("After switch to 1-hot encoding for disease feature")
print("New Shape of Inputs: X =", cleaned_X.shape)
print("New Shape of Outputs: Y =", cleaned_Y.shape)
print("")

# Display the modified DataFrame
print("Modified Inputs: ")
print(cleaned_X)

Original Dataset: 
Shape of Inputs: X = (187, 36)
Shape of Outputs: Y = (187, 1)

Redudant and conflicting features: 
 -- time to development of agvhd stage iii and iv
 -- survival time
 -- recipientage10

After removal of redundant and conflicting features, and patients with missing data: 
New Shape of Inputs: X = (142, 33)
New Shape of Outputs: Y = (142, 1)

After switch to 1-hot encoding for disease feature
New Shape of Inputs: X = (142, 36)
New Shape of Outputs: Y = (142, 1)

Modified Inputs: 
     recipientgender  stemcellsource   donorage  donorage35  iiiv  \
0                1.0             1.0  22.830137         0.0   1.0   
1                1.0             0.0  23.342466         0.0   1.0   
2                1.0             0.0  26.394521         0.0   1.0   
3                0.0             1.0  33.358904         0.0   0.0   
4                1.0             1.0  32.641096         0.0   0.0   
..               ...             ...        ...         ...   ...   
137           

In [ ]:
# viewing modified input. looking for any categories that require preprocressing --> since unable to view more than 20 columns of pandas dataframe
readX = cleaned_X.to_csv("inputsFinal.csv")
readY = cleaned_Y.to_csv("outputsFinal.csv")


In [ ]:
# Ensuring all data types are float
cleaned_X = cleaned_X.astype('float')
cleaned_Y = cleaned_Y.astype('float')

# Convert DataFrames to numpy array without column names --> prep for computations
X_array = cleaned_X.values; print(X_array.shape)
Y_array = cleaned_Y.values; print(Y_array.shape)
# print(X_array, " ", X_array.shape); print(); print(Y_array, " ", Y_array.shape) # check arrays


(142, 36)
(142, 1)


# <font color= 'blue'> Preliminary Model Architecture
Model supports entropy calculation (log 2 --> in bits), activation functions (sigmoid vs. reLU), loss terms (sum of squares vs cross-entropy).
First ran on XOR function to verify

In [ ]:
# aesthetics & other imports
import numpy as np

class color:
  MAGENTA = '\033[35;48m'
  CYAN = '\033[36;48m'
  BOLD = '\033[37;48m'
  BLUE = '\033[34;48m'
  GREEN = '\033[32;48m'
  YELLOW = '\033[33;48m'
  RED = '\033[91;48m'
  BLACK = '\033[30;48m'
  BOLD = '\033[1;30;48m' # (bolded BLACK)
  UNDERLINE = '\033[4;37;48m'
  END = '\033[1;37;0m'


In [ ]:
# we will input a list of numbers of neurons to specify neurons in each layer (len(layers) = number of layers)
# e.g. layers = [3, 6, 3] -- 3 layers: 3 neurons, 6 neurons, 3 neurons

class FeedforwardNeuralNetwork:
  # specified activation for output layer

  def __init__(self, layers, alpha = 0.1, activation='sigmoid', output_activation='sigmoid', loss="sum-of-squares"):
    self.W = [] # list of matrices per layer
    self.layers = layers
    self.alpha = alpha
    self.activation = activation
    self.output_activation = output_activation
    self.loss = loss
    self.lossList = [] # list of loss per epoch
    self.epochEntropies = [] # list of total entropies per epoch (all layers normalized and additive) --> should it be normalized? probably not. doesnt need to be. just total entropy.
    self.layerEntropies = [] # list of lists of entropies per layer (goes by layer. 1st layer exempt) Note: ONLY FOR LAST EPOCH!!


    # initialize the weights between layers (up to the next-to-last one) as random
    for i in np.arange(0, len(layers) - 2): # np.arrange is the same thing as range
      currentNeurons = layers[i]
      nextNeurons = layers [i+1]
      self.W.append(np.random.randn(currentNeurons + 1, nextNeurons + 1)) # +1 is for the bias neurons # random.randn --> mean=0, standard deviation=1

    # initialize weights between the last two layers --> dont want bias for last layer!!
    self.W.append(np.random.randn(layers[-2] + 1, layers[-1]))

  def activate(self, layer, x): # layer starts with index 0 and includes input layer
  # SOFTMAX IS STRICTLY ONLY IN OUTPUT ACTIVATION LAYER

    if layer != (len(self.layers)-1):
      if self.activation == 'sigmoid':
        return 1.0 / (1 + np.exp(-x))
      if self.activation == 'reLU':
        return np.maximum(0, x)

    else: # means we are in the output layer
      if self.output_activation == 'sigmoid':
        return 1.0 / (1 + np.exp(-x))
      if self.output_activation == 'reLU':
        return np.maximum(0, x)

  def activation_Derivative(self, layer, z): # layer starts with index 0 and includes input layer
    # SOFTMAX IS STRICTLY ONLY IN OUTPUT ACTIVATION LAYER
    if layer != (len(self.layers)-1):
      if self.activation == 'sigmoid':
        return z * (1 - z)
      if self.activation == 'reLU':
        return np.heaviside(z, 1)
    else: # means we are in the output layer
      if self.output_activation == 'sigmoid':
        return z * (1 - z)
      if self.output_activation == 'reLU':
        return np.heaviside(z, 1)

  def compute_entropy(self, layer, activationList): # layer has to include input layer and start with index 0
    entropy = 0.0
    activate_type = ""

    if layer != (len(self.layers)-1): # if not last layer
      activate_type = self.activation
    else:
      activate_type = self.output_activation

    # if activate_type == 'reLU': --> entropy is 0, calculation not possible

    if activate_type == 'sigmoid':
      for i in range(len(activationList)): # want number of activations in activationList
        entropy += (activationList[i] * np.log2(activationList[i])) # sum of (activations * log of activations). log2 for bits
        entropy = -entropy # negative sign

    return entropy


  # fit the model
  def fit(self, X, y, epochs = 10000, update = 1000):
    # add a column of ones to the end of X
    X = np.hstack((X, np.ones([X.shape[0],1]))) # --> also automatically turns X into an array & ignores columns

    # list of lists per layer
    epoch_totalList = []

    for epoch in np.arange(0, epochs):
      master_entropyList = [[] for i in range(len(self.layers)-1)] # entropyList for each layer to equal list of entropies for each epoch. -1 because first layer does not have calculated entropy since it is an input layer
      # feed forward, backprop, and weight update
      for (x, target) in zip(X, y):

        # make a list of output activations from the first layer
        A = [np.atleast_2d(x)] # currently --> (just the original x values)

        # feed forward
        for layer in np.arange(0, len(self.W)): # for each layer in weights list, which does not include the first nodes because those are just the inputs. those nodes are not really neurons, they only calculate inputs*weights, and then the activation that occurs will be applied by the second layer's neurons
          neuronsNum = self.layers[layer+1]

          # feed through one layer and apply sigmoid activation
          z = A[layer].dot(self.W[layer]) # multiplying layer inputs with weights # .dot is the same as @ --> matrix multiplication
          out = self.activate(layer+1, z) # activation function being applied

          # add our network output to the list of activations --> append activation to A

          A.append(out) # storing activation
          insideOut = out[len(out)-1] # since out is a list of lists.
          entropy_perLay_perInp = self.compute_entropy(layer+1, insideOut) # entropy per layer per input --> no normalization
          master_entropyList[layer].append(entropy_perLay_perInp)
          # calculating entropy for the layer

        # backpropagation

        error = A[-1] - target # sum of least squares error, where A[-1] is the activation from the last layer (output)
        # only get error from last layer because it corresponds to the output/prediction, which can only be obtained from last layer?? (i think)


        # term proportional to the gradient --> make a list to store the derivatives
        last_lay_index = len(self.layers)-1
        D = [error * self.activation_Derivative(last_lay_index, A[-1])] # can change this to use cross-entropy error instead of sum of squared errors

        # cross-entropy only works for classification

        # loop backwards over the layers to build up deltas --> backpropagate for errors
        for layer in np.arange(len(A) - 2, 0, -1): # basicaly excludes input and output layer
          delta = D[-1].dot(self.W[layer].T)
          delta = delta * self.activation_Derivative(layer+1, A[layer])
          D.append(delta)

        # reverse the deltas since we looped in reverse
        D = D[::-1] # now D contains all of the partial derivatives

        # weight update
        for layer in np.arange(0, len(self.W)):
          self.W[layer] -= self.alpha * A[layer].T.dot(D[layer])

      totalEpochEnt = 0
      for i in range(len(master_entropyList)): # accessing master list --> i is layer #. layer starts at the second layer, since first layer is input layer (no calculation for input layer)
        summationEnt = 0

        for j in range(len(master_entropyList[i])): # j is input # for ith layer. trying to compile all input entropies and average them for each layer (layer goes by i)

          entropy_perLayList = master_entropyList[i]
          summationEnt += entropy_perLayList[j]

          if j == (len(entropy_perLayList)-1):

            summationAvg = summationEnt / len(entropy_perLayList) # len(entropy_perLayList = number of x's (input rows) in X)
            totalEpochEnt += summationAvg
            master_entropyList[i] = summationAvg/(self.layers[i+1]) # dividing total entropy for particular layer by its number of neurons

      if epoch == epochs-1:
        self.layerEntropies = master_entropyList # storing last and final epoch's individual layer entropies
        print("Note: The following entropies stated are the averages found only in the last epoch, which is epoch ", epoch)
        print("If the entropy is 0, then most likely the entropy is not able to be calculated for the specific activation function used in the layer")
        for i in range(len(master_entropyList)):
          print(color.MAGENTA, "Average total entropy for layer (Normalized)", i+2, ": ", master_entropyList[i], color.END, "Info: Layer had ", self.layers[i+1], " neurons. ")

      self.epochEntropies.append(totalEpochEnt) # storing total entropy from all layers from epoch (average across inputs)

      # print a status update
      if (epoch + 1) % update == 0:
        loss = self.computeLoss(X,y)
        self.lossList.append(loss)
        print(color.BLUE, 'EPOCH =', epoch + 1, 'LOSS = ', loss, color.END)

  def predict(self, X, addOnes = True):
    # initialize data, be sure it's the right dimension
    p = np.atleast_2d(X)

    # add a column of 1s for bias
    if addOnes: # in case we already did appending before we called the predict function!
      p = np.hstack((p, np.ones([X.shape[0],1])))

    # feed forward!
    for layer in np.arange(0, len(self.W)):
      p = self.activate(layer+1, np.dot(p, self.W[layer])) # only need final output, dont need to store all of it
      # final p will be the final predictions that were outputted from the last layer of the neural network

    # return the predictions
    return p

  def computeLoss(self, X, y):
    # initialize data, be sure it's the right dimension
    y = np.atleast_2d(y) # might not need this

    # feed the datapoints through the network to get predicted outputs
    predictions = self.predict(X, addOnes = False)

    # compute the sum of squared errors loss function
    if(self.loss == "sum-of-squares"):
      loss = np.sum((predictions - y)**2) / 2.0

    # compute cross entropy loss function. it is more effective because it uses a log so it more heavily penalizes bad predictions
    if(self.loss == "cross-entropy"):
      loss = np.sum(np.nan_to_num(-y * np.log(predictions) - (1 - y) * np.log(1 - predictions)))

    return loss


## <font color= 'blue'> Finally doing a preliminary test on the data!!

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

# split the data into training and test sets (randomized)
trainX, testX, trainY, testY = train_test_split(X_array, Y_array, test_size = 0.25, random_state = 1)

trainX = scale(trainX)
testX = scale(testX) # have to scale to make sure parameters are scaled the same in both --> question: how is it being scaled?

print("trainInputs shape: ", trainX.shape); print("trainOutputs shape: ", trainY.shape)
print("testInputs shape: ", testX.shape); print("testInputs shape: ", testY.shape)


trainInputs shape:  (106, 36)
trainOutputs shape:  (106, 1)
testInputs shape:  (36, 36)
testInputs shape:  (36, 1)


# for relu with output activation as sigmoid, it is quite suspicious how the output is either 0.5 or lower. also inf is being encountered upon many epochs because loss becomes exponential

In [ ]:

# mini-reLU
print("for reLU: ")
model = FeedforwardNeuralNetwork(layers = [36, 1, 1], loss='cross-entropy', activation='reLU', output_activation='sigmoid') # 3 layers, have to have 37 input "nodes" in first layer, and 1 output "nodes" in last layer
model.fit(trainX, trainY, 200, 50)
#model.fit(trainX, trainY, 2, 1) # making sure there are no overflow errors
print("")
model.predict(testX)
predictions = model.predict(testX)


print("")

print("non interpreted version: ")
print(predictions)
for i in range(len(predictions)):
  if predictions[i] < 0.50000000000001: # works if interpretation is < 0.5
    predictions[i] = 0
  else:
    predictions[i] = 1
print("interpreted version: ")
print(predictions)

#print(predictions)
print()
print("the true values are: ")
print(testY)
print("Seeing if predictions worked: ")
for i in range(len(predictions)):
  if predictions[i] == testY[i]:
    print("True")
  else:
    print("False, answer was: ", testY[i])

# ReLU does not work well with cross-entropy loss because it does not always output a probability for activation


for reLU: 
 EPOCH = 50 LOSS =  1453.9171801597324 


<ipython-input-38-fe06c17386fc>:191: RuntimeWarning: divide by zero encountered in log
  loss = np.sum(np.nan_to_num(-y * np.log(predictions) - (1 - y) * np.log(1 - predictions)))


 EPOCH = 100 LOSS =  1.7976931348623157e+308 


<ipython-input-38-fe06c17386fc>:191: RuntimeWarning: invalid value encountered in multiply
  loss = np.sum(np.nan_to_num(-y * np.log(predictions) - (1 - y) * np.log(1 - predictions)))
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 EPOCH = 150 LOSS =  inf 
Note: The following entropies stated are the averages found only in the last epoch, which is epoch  199
If the entropy is 0, then most likely the entropy is not able to be calculated for the specific activation function used in the layer
 Average total entropy for layer (Normalized) 2 :  0.0  Info: Layer had  1  neurons. 
 Average total entropy for layer (Normalized) 3 :  0.06018984962076109  Info: Layer had  1  neurons. 
 EPOCH = 200 LOSS =  inf 


non interpreted version: 
[[9.99999956e-01]
 [3.24038114e-22]
 [3.46185298e-34]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [6.83149902e-30]
 [1.00000000e+00]
 [1.98190208e-54]
 [6.11190948e-13]
 [2.72456067e-07]
 [9.99941721e-01]
 [9.67500159e-03]
 [3.88023993e-18]
 [9.99999230e-01]
 [3.79903918e-09]
 [1.65096657e-29]
 [1.00000000e+00]
 [1.67678048e-29]
 [9.99989528e-01]
 [1.37082086e-15]
 [2.53320346e-48]
 [1.61835151e-18]
 [3.83786609e-05]
 [1.30029092e-26]
 [1.00000000e+00]
 [6.61392106e-25]
 [5.0000

In [ ]:
# CROSS ENTROPY LOSS DOES NOT WORK WITH reLU !!!
# mini-reLU
print("for reLU: ")
model = FeedforwardNeuralNetwork(layers = [36, 1, 1], loss='sum-of-squares', activation='reLU', output_activation='sigmoid') # 3 layers, have to have 37 input "nodes" in first layer, and 1 output "nodes" in last layer
model.fit(trainX, trainY, 2, 1)
print("")

# print(testX)
predictions = model.predict(testX)
print("non interpreted version: ")
print(predictions)
for i in range(len(predictions)):
  if predictions[i] < 0.50000000001:
    predictions[i] = 0
  else:
    predictions[i] = 1
print("interpreted version: ")
print(predictions)

#print(predictions)
print()
print("the true values are: ")
print(testY)
print("Seeing if predictions worked: ")
for i in range(len(predictions)):
  if predictions[i] == testY[i]:
    print("True")
  else:
    print("False, answer was: ", testY[i])

# ReLU does not work well with cross-entropy loss because it does not always output a probability for activation


for reLU: 
 EPOCH = 1 LOSS =  12.50042669826295 
Note: The following entropies stated are the averages found only in the last epoch, which is epoch  1
If the entropy is 0, then most likely the entropy is not able to be calculated for the specific activation function used in the layer
 Average total entropy for layer (Normalized) 2 :  0.0  Info: Layer had  1  neurons. 
 Average total entropy for layer (Normalized) 3 :  0.33296685863936853  Info: Layer had  1  neurons. 
 EPOCH = 2 LOSS =  14.372622456280506 

non interpreted version: 
[[2.34742742e-02]
 [5.00000000e-01]
 [5.00000000e-01]
 [2.28785159e-01]
 [6.89842746e-03]
 [4.88635841e-03]
 [2.33239770e-01]
 [2.26428743e-01]
 [5.00000000e-01]
 [8.83955025e-03]
 [5.00000000e-01]
 [2.06656008e-02]
 [8.06599820e-04]
 [1.05490810e-01]
 [1.82070908e-01]
 [1.86225030e-03]
 [5.00000000e-01]
 [6.37915382e-04]
 [1.65426702e-03]
 [1.93098010e-02]
 [5.33830097e-03]
 [1.56935840e-02]
 [3.91833979e-01]
 [3.83571457e-01]
 [1.46004152e-03]
 [1.0916605

# ReLU with 4 layers --> need to see how to address vanishing / exploding gradients. overflow is being encountered at sigmoid output activation

In [ ]:
# CROSS ENTROPY LOSS DOES NOT WORK WITH reLU !!!
# mini-reLU
print("for reLU with 4 layers: ")
model = FeedforwardNeuralNetwork(layers = [36, 18, 9, 1], loss='sum-of-squares', activation='reLU', output_activation='sigmoid') # 3 layers, have to have 37 input "nodes" in first layer, and 1 output "nodes" in last layer
model.fit(trainX, trainY, 2, 1)
print("")

# print(testX)
predictions = model.predict(testX)
print("non interpreted version: ")
print(predictions)
for i in range(len(predictions)):
  if predictions[i] < 0.50000000001:
    predictions[i] = 0
  else:
    predictions[i] = 1
print("interpreted version: ")
print(predictions)

#print(predictions)
print()
print("the true values are: ")
print(testY)
print("Seeing if predictions worked: ")
for i in range(len(predictions)):
  if predictions[i] == testY[i]:
    print("True")
  else:
    print("False, answer was: ", testY[i])

# ReLU does not work well with cross-entropy loss because it does not always output a probability for activation


for reLU with 4 layers: 
 EPOCH = 1 LOSS =  17.49999999990724 
Note: The following entropies stated are the averages found only in the last epoch, which is epoch  1
If the entropy is 0, then most likely the entropy is not able to be calculated for the specific activation function used in the layer
 Average total entropy for layer (Normalized) 2 :  0.0  Info: Layer had  18  neurons. 
 Average total entropy for layer (Normalized) 3 :  0.0  Info: Layer had  9  neurons. 
 Average total entropy for layer (Normalized) 4 :  nan  Info: Layer had  1  neurons. 
 EPOCH = 2 LOSS =  17.49999999990789 

non interpreted version: 
[[1.06094966e-064]
 [2.20654307e-040]
 [2.10468400e-234]
 [0.00000000e+000]
 [0.00000000e+000]
 [0.00000000e+000]
 [1.18658106e-164]
 [0.00000000e+000]
 [0.00000000e+000]
 [2.02499902e-193]
 [1.41036203e-055]
 [0.00000000e+000]
 [0.00000000e+000]
 [3.11175678e-289]
 [2.85160668e-276]
 [1.26917434e-163]
 [0.00000000e+000]
 [1.09034660e-101]
 [0.00000000e+000]
 [6.17719457e-23

<ipython-input-38-fe06c17386fc>:39: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-x))
<ipython-input-38-fe06c17386fc>:69: RuntimeWarning: divide by zero encountered in log2
  entropy += (activationList[i] * np.log2(activationList[i])) # sum of (activations * log of activations). log2 for bits
<ipython-input-38-fe06c17386fc>:69: RuntimeWarning: invalid value encountered in scalar multiply
  entropy += (activationList[i] * np.log2(activationList[i])) # sum of (activations * log of activations). log2 for bits


# doing calculations with sigmoid!!

In [ ]:
print("for basic network: ")
# LOSS --> SUM OF SQUARES
# SIGMOID ACTIVATIONS AND SIGMOID OUTPUT ACTIVATIONS
model = FeedforwardNeuralNetwork(layers = [36, 1, 1]) # 3 layers, have to have 37 input "nodes" in first layer, and 1 output "nodes" in last layer
model.fit(trainX, trainY, 1000, 100) # making sure there are no overflow errors
print("")

print(testX)
predictions = model.predict(testX)
print("non interpreted version: ")
print(predictions)
for i in range(len(predictions)):
  if predictions[i] < 0.5:
    predictions[i] = 0
  else:
    predictions[i] = 1
print("interpreted version: ")
print(predictions)

#print(predictions)
print()
print("the true values are: ")
print(testY)
print("Seeing if predictions worked: ")
for i in range(len(predictions)):
  if predictions[i] == testY[i]:
    print("True")
  else:
    print("False, answer was: ", testY[i])




for basic network: 
 EPOCH = 100 LOSS =  9.359234811338407 
 EPOCH = 200 LOSS =  6.975886518954928 
 EPOCH = 300 LOSS =  6.484089065867445 
 EPOCH = 400 LOSS =  6.371906276104341 
 EPOCH = 500 LOSS =  6.317212572130562 
 EPOCH = 600 LOSS =  6.27219729500222 
 EPOCH = 700 LOSS =  6.222197801234125 
 EPOCH = 800 LOSS =  5.5238863040123665 
 EPOCH = 900 LOSS =  5.418052579250774 
Note: The following entropies stated are the averages found only in the last epoch, which is epoch  999
If the entropy is 0, then most likely the entropy is not able to be calculated for the specific activation function used in the layer
 Average total entropy for layer (Normalized) 2 :  0.053361545948121175  Info: Layer had  1  neurons. 
 Average total entropy for layer (Normalized) 3 :  0.23511621395485127  Info: Layer had  1  neurons. 
 EPOCH = 1000 LOSS =  5.212303083779949 

[[ 0.75180941  0.35355339  0.81377978 ... -1.18321596 -0.75180941
   0.79772404]
 [ 0.75180941  0.35355339 -1.561873   ... -1.18321596 

In [ ]:
print("for 4 layers basic neural network: ")
print("all sigmoid activations and sum of squares loss")
# LOSS --> SUM OF SQUARES
# SIGMOID ACTIVATIONS AND SIGMOID OUTPUT ACTIVATIONS
model = FeedforwardNeuralNetwork(layers = [36, 18, 9,  1]) # 3 layers, have to have 37 input "nodes" in first layer, and 1 output "nodes" in last layer
model.fit(trainX, trainY, 500, 100) # making sure there are no overflow errors
print("")

# print(testX)
predictions = model.predict(testX)
print("non interpreted version: ")
print(predictions)
for i in range(len(predictions)):
  if predictions[i] < 0.5:
    predictions[i] = 0
  else:
    predictions[i] = 1

#print(predictions)
print()
#print("the true values are: ")
print(testY)
print("Seeing if predictions worked: ")
for i in range(len(predictions)):
  if predictions[i] == testY[i]:
    print("True")
  else:
    print("False, answer was: ", testY[i])

for 4 layers basic neural network: 
 EPOCH = 100 LOSS =  0.2116806674020649 
 EPOCH = 200 LOSS =  0.0626595321399891 
 EPOCH = 300 LOSS =  0.03436943769347317 
 EPOCH = 400 LOSS =  0.023131917230458453 
Note: The following entropies stated are the averages found only in the last epoch, which is epoch  499
If the entropy is 0, then most likely the entropy is not able to be calculated for the specific activation function used in the layer
 Average total entropy for layer (Normalized) 2 :  0.006795591680520307  Info: Layer had  18  neurons. 
 Average total entropy for layer (Normalized) 3 :  -0.002644607537765477  Info: Layer had  9  neurons. 
 Average total entropy for layer (Normalized) 4 :  0.05856906477004634  Info: Layer had  1  neurons. 
 EPOCH = 500 LOSS =  0.017235085729944196 

[[ 0.75180941  0.35355339  0.81377978 ... -1.18321596 -0.75180941
   0.79772404]
 [ 0.75180941  0.35355339 -1.561873   ... -1.18321596 -0.75180941
  -1.25356634]
 [ 0.75180941  0.35355339  0.12207873 ...  

In [ ]:
print("for 5 layers basic neural network: ")
print("all sigmoid activations & sum of squares loss")
# LOSS --> SUM OF SQUARES
# SIGMOID ACTIVATIONS AND SIGMOID OUTPUT ACTIVATIONS
model = FeedforwardNeuralNetwork(layers = [36, 20, 18, 9, 1]) # 3 layers, have to have 37 input "nodes" in first layer, and 1 output "nodes" in last layer
model.fit(trainX, trainY, 500, 100) # making sure there are no overflow errors
print("")

print(testX)
predictions = model.predict(testX)
print("non interpreted version: ")
print(predictions)
for i in range(len(predictions)):
  if predictions[i] < 0.5:
    predictions[i] = 0
  else:
    predictions[i] = 1

#print(predictions)
print()
#print("the true values are: ")
print(testY)
print("Seeing if predictions worked: ")
for i in range(len(predictions)):
  if predictions[i] == testY[i]:
    print("True")
  else:
    print("False, answer was: ", testY[i])

for 5 layers basic neural network: 
 EPOCH = 100 LOSS =  0.2335769213340288 
 EPOCH = 200 LOSS =  0.05357457077373107 
 EPOCH = 300 LOSS =  0.02763618588854496 
 EPOCH = 400 LOSS =  0.018092661896134767 
Note: The following entropies stated are the averages found only in the last epoch, which is epoch  499
If the entropy is 0, then most likely the entropy is not able to be calculated for the specific activation function used in the layer
 Average total entropy for layer (Normalized) 2 :  0.017749325399780812  Info: Layer had  20  neurons. 
 Average total entropy for layer (Normalized) 3 :  0.03770047892091271  Info: Layer had  18  neurons. 
 Average total entropy for layer (Normalized) 4 :  -0.008410121696781386  Info: Layer had  9  neurons. 
 Average total entropy for layer (Normalized) 5 :  0.05746917421764185  Info: Layer had  1  neurons. 
 EPOCH = 500 LOSS =  0.013270786753253843 

[[ 0.75180941  0.35355339  0.81377978 ... -1.18321596 -0.75180941
   0.79772404]
 [ 0.75180941  0.353

In [ ]:
# mini-cross entropy
print("sigmoid for all layers and cross entropy loss: ")
model = FeedforwardNeuralNetwork(layers = [36, 1, 1], loss='cross-entropy') # 3 layers, have to have 37 input "nodes" in first layer, and 1 output "nodes" in last layer
model.fit(trainX, trainY, 10000, 1000) # making sure there are no overflow errors
print("")

print(testX)
predictions = model.predict(testX)
print("non interpreted version: ")
print(predictions)
for i in range(len(predictions)):
  if predictions[i] < 0.5:
    predictions[i] = 0
  else:
    predictions[i] = 1

#print(predictions)
print()
#print("the true values are: ")
print(testY)
print("Seeing if predictions worked: ")
for i in range(len(predictions)):
  if predictions[i] == testY[i]:
    print("True")
  else:
    print("False, answer was: ", testY[i])

for cross entropy: 
 EPOCH = 1000 LOSS =  21.526664831747755 
 EPOCH = 2000 LOSS =  16.603056188576623 
 EPOCH = 3000 LOSS =  15.339225822273722 
 EPOCH = 4000 LOSS =  14.231757047536824 
 EPOCH = 5000 LOSS =  13.790998928825378 
 EPOCH = 6000 LOSS =  13.63968219016571 
 EPOCH = 7000 LOSS =  13.514362678630446 
 EPOCH = 8000 LOSS =  12.855642106194708 
 EPOCH = 9000 LOSS =  12.751601718329685 
Note: The following entropies stated are the averages found only in the last epoch, which is epoch  9999
If the entropy is 0, then most likely the entropy is not able to be calculated for the specific activation function used in the layer
 Average total entropy for layer (Normalized) 2 :  0.021863304410239346  Info: Layer had  1  neurons. 
 Average total entropy for layer (Normalized) 3 :  0.11323789655101762  Info: Layer had  1  neurons. 
 EPOCH = 10000 LOSS =  12.687502763245046 

[[ 0.75180941  0.35355339  0.81377978 ... -1.18321596 -0.75180941
   0.79772404]
 [ 0.75180941  0.35355339 -1.56187

In [ ]:
# data for exclusive OR function in coding
# e.g. [0, 0] is a feature vector of inputs, and [0] is its corresponding output

X1 = np.array([[0, 0], [1, 0], [0, 1], [1, 1]], dtype=float)
Y1 = np.array([[0], [1], [1], [0]], dtype=float)


In [ ]:
model = FeedforwardNeuralNetwork(layers = [2, 2, 1])
# neurons in first layer is determined by the dimension of the input which is 2
# neurons in the hidden layer can be however many you want
# neurons in the last layer is determined by the dimension of the output which is 1

vars(model)


{'W': [array([[-0.46903939, -0.04988735, -1.47060113],
         [ 0.53458613,  0.42089307, -2.03731276],
         [ 1.28074014,  1.73331365, -1.38888897]]),
  array([[ 2.42674676],
         [-1.16603755],
         [ 0.07984697]])],
 'layers': [2, 2, 1],
 'alpha': 0.1,
 'activation': 'sigmoid',
 'output_activation': 'sigmoid',
 'loss': 'sum-of-squares',
 'lossList': [],
 'epochEntropies': [],
 'layerEntropies': []}

In [ ]:
print(model.W[0].shape)
print(model.W[1].shape)
print(len(model.W)) # only 2 weight matrices even though there are 3 layers. because output of last layer is the prediction, it does not use another weight matrix.


(3, 3)
(3, 1)
2


In [ ]:
model.fit(X1, Y1, 10000, 1000)

# at 1:02:43 of lecture 8 - "implementing backpropagation"


 EPOCH = 1000 LOSS =  0.1095410223979115 
 EPOCH = 2000 LOSS =  0.034246885912749325 
 EPOCH = 3000 LOSS =  0.01770399692754239 
 EPOCH = 4000 LOSS =  0.011463656541701615 
 EPOCH = 5000 LOSS =  0.008331880014235436 
 EPOCH = 6000 LOSS =  0.006486014917034683 
 EPOCH = 7000 LOSS =  0.005281815551439799 
 EPOCH = 8000 LOSS =  0.0044396668389452875 
 EPOCH = 9000 LOSS =  0.0038202476008797294 
Note: The following entropies stated are the averages found only in the last epoch, which is epoch  9999
If the entropy is 0, then most likely the entropy is not able to be calculated for the specific activation function used in the layer
 Average total entropy for layer (Normalized) 2 :  0.08614103679160928  Info: Layer had  2  neurons. 
 Average total entropy for layer (Normalized) 3 :  0.12207909951840301  Info: Layer had  1  neurons. 
 EPOCH = 10000 LOSS =  0.0033469119378694775 


In [ ]:
model.predict(X1) # model works for classic XOR function!!


array([[0.03761742],
       [0.97455371],
       [0.94938057],
       [0.0454853 ]])